<a href="https://colab.research.google.com/github/cszubert/colab/blob/master/Hands_on_ML_Workbook_Hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.13.0'

In [2]:
tf.random.set_seed(13)

### Load California Housing

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [11]:
X_new = X_test[:3]

### Hyperparameters

In [4]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation='relu'))
  model.add(keras.layers.Dense(1))
  #optimizer = keras.optimizers.SGD(lr=learning_rate)
  # WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate`
  optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  model.compile(loss='mse', optimizer=optimizer)
  return model

#### Hyperparameter tunning with KerasRegressor

In [5]:
# keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
## NOTE: keras.wrappers.scikit_learn had been removed

Migrating from tf.keras.wrappers.scikit_learn: https://adriangb.com/scikeras/stable/migration.html

In [6]:
! pip install scikeras

In [7]:
import scikeras
from scikeras.wrappers import KerasRegressor
scikeras.__version__

'0.12.0'

In [8]:
keras_reg = KerasRegressor(build_model)

In [13]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 2s 4ms/step - loss: 1.2868 - val_loss: 1.5400
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5953 - val_loss: 0.6233
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5235 - val_loss: 0.4697
Epoch 4/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4874 - val_loss: 0.4749
Epoch 5/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4648 - val_loss: 0.4217
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4508 - val_loss: 0.4224
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4400 - val_loss: 0.4505
Epoch 8/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4323 - val_loss: 0.3953
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4249 - val_loss: 0.3880
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4192 - val_lo

KerasRegressor(
	model=<function build_model at 0x78c1d358a170>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
)

In [14]:
y_pred = keras_reg.predict(X_new)
y_pred

1/1 [==============================] - 0s 36ms/step


array([0.5994873, 1.7632403, 4.256681 ], dtype=float32)

In [15]:
mse_test = keras_reg.score(X_test, y_test)
mse_test

162/162 [==============================] - 0s 1ms/step


0.7447060068116318